# Backpropagation Part 2

## Softmax

In [16]:
import numpy as np
from tensorflow.keras.layers import Softmax, Conv2D, Dense, Input
from tensorflow.keras.losses import CategoricalCrossentropy

In [ ]:
softmax = Softmax()


def my_softmax(y):
    z = np.sum(np.exp(y), axis=1, keepdims=True)
    log_probs = y - np.log(z)
    return np.exp(log_probs)

def my_better_softmax(y):
    shifted_logits = y - np.max(y, axis=1, keepdims=True)
    z = np.sum(np.exp(shifted_logits), axis=1, keepdims=True)
    log_probs = shifted_logits - np.log(z)
    return np.exp(log_probs)

In [ ]:
y = np.array([[-200, 100, 5], [-200, 100, -5]]).astype(np.float64) * 1000
print(softmax(y))
print(my_softmax(y))
print(my_better_softmax(y))

tf.Tensor(
[[0. 1. 0.]
 [0. 1. 0.]], shape=(2, 3), dtype=float32)
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 1. 0.]
 [0. 1. 0.]]


/tmp/ipykernel_358000/838545724.py:5: RuntimeWarning: overflow encountered in exp
  z = np.sum(np.exp(y), axis=1, keepdims=True)


## Cross Entropy

In [ ]:
y_true = np.array([[0.3, 0.4, 0.3], [0.5, 0.5, 0.0], [1.0, 0.0, 0.0]])
y_pred = np.array([[0.3, 0.4, 0.29], [0.5, 0.5, 0.01], [0.99, 0.01, 0.01]])
cce = CategoricalCrossentropy()
cce(y_true, y_pred).numpy()

0.6040394277047364

In [ ]:
def my_cross_entropy(y_true, y_pred):
    log_probs = np.log(y_pred)
    return -np.sum(y_true * log_probs) / y_true.shape[0]

my_cross_entropy(y_true, y_pred)

0.6007559857537915

## Combine Everything

In [1]:
import numpy as np
from layers.dense import Dense
from layers.activations import ReLU
from IPython.display import Code

In [5]:
Code(filename='layers/dense.py', language='python')

import numpy as np


class Dense:
    def __init__(self):
        self.x = None
        self.w = None
        self.b = None

    def __call__(self, x, w, b):
        """Perform forward propagation

        Args:
            x (np.ndarray): Input
            w (np.ndarray): Kernel weights
            b (np.ndarray): Biases

        Returns:
            np.ndarray: Output
        """
        self.x = x
        self.w = w
        self.b = b
        return np.matmul(x, w) + b

    def gradients(self, grad_out):
        """Perform back propagation and return gradients with respect to upstream loss function.

        Args:
            grad_out (np.ndarray): Gradient of loss with respect to output.

        Returns:
            np.ndarray: Gradient of loss with respect to x
            np.ndarray: Gradient of loss with respect to w
            np.ndarray: Gradient of loss with respect to b
        """
        if self.x is None:
            raise ValueError("layer must be forward propagated first")
        
        grad_x = np.matmul(grad_out, self.w.T)
        grad_w = np.matmul(self.x.T, grad_out)
        grad_b = np.sum(grad_out, axis=0)
        return grad_x, grad_w, grad_b